In [3]:
import re
import os
import sys
import time
import warnings
import datetime
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

warnings.filterwarnings(action='ignore')

from tqdm import tqdm
tqdm.pandas()

import torch
from kobart import get_pytorch_kobart_model, get_kobart_tokenizer
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration, BartModel
from rouge import Rouge

In [4]:
df = pd.read_csv('조경뉴스.csv')

In [5]:
df.head()

,title,sub-title,publisher,author,date,text,keywords,href,numKeywords,news_id
0,"[캠퍼스] 한국전통문화대 전통조경학과, 고택정원 7곳 평면도 작성 실시",고택정원의 과거와 현재의 차이점 연구,라펜트,김태원,2019-04-19,한국전통문화대학교 전통조경학과 2학년 학생들이 이재용 교수의 ‘전통식재론’ 수업의 ...,"['한국전통문화대학교', '이재용 교수', '전통조경', '3D 활용', '조경학과']",https://www.lafent.com/inews/news_view.html?ne...,5,124048
1,2022년 올해의 정원식물은?,‘2022 대한민국 정원식물 전시·품평회’ 성료,라펜트,전지은,2022-11-07,‘2022 대한민국 정원식물 전시·품평회’를 9월 30일부터 10월 30일까지 개...,"['한국수목원정원관리원', '대한민국 정원식물 전시·품평회', '정원식물']",https://www.lafent.com/inews/news_view.html?ne...,3,131470
2,‘골목길’ 주제 도시재생세미나 30일 개최,김선아 대표·김영욱 교수 주제발표,한국건설신문,선태규,2020-01-17,한국건축가협회 도시재생위원회위원장 김선아는 오는 30일 오후 6시 30분부터 8시 ...,"['한국건축가협회', '도시재생위원회', '도시재생', '골목길']",https://www.lafent.com/inews/news_view.html?ne...,4,125638
3,03조경_ 옴스테드와 보의 사잇생각: 테크네 ‘조경’(上),"경공환장: 다시 보는 일상, 느껴 보는 도시_3회",라펜트,안명준,2013-02-08,경공환장景空環場\n다시 보는 일상 느껴 보는 도시\n03조경 옴스테드와 보의 사잇생...,"['경공환장', '안명준', '옴스테드', '조경']",https://www.lafent.com/inews/news_view.html?ne...,4,108511
4,"익산시, 철도 유휴부지 활용 본격화…철도공단과 협력",송학동~오산면 구간 2025년까지 기후대응 도시숲 7ha 조성,라펜트,주선영,2023-07-11,﻿철도 유휴부지 활용 예시 익산시 제공 익산 송학동과 오산면을 잇는 장항선 폐철도...,"['익산시', '국가철도공단', '장항선', '폐철도 유휴부지', '도시숲', '녹지']",https://www.lafent.com/inews/news_view.html?ne...,6,132563


BART(Bidirectional and Auto-Regressive Transformers)는 입력 텍스트 일부에 노이즈를 추가하여 이를 다시 원문으로 복구하는 autoencoder의 형태로 학습이 됩니다. 한국어 BART(이하 KoBART) 는 논문에서 사용된 `Text Infilling` 노이즈 함수를 사용하여 40GB 이상의 한국어 텍스트에 대해서 학습한 한국어 `encoder-decoder` 언어 모델입니다. 이를 통해 도출된 `KoBART-base`를 배포합니다.

문석 요약 두 가지 종류:
- 생성요약(abstractive summary) - 토큰 간의 관계를 계산하여 원래 문서에 포함되지 않은 문장으로 요약
- 추출요약(extractive summary) - 토큰 간의 관계를 계산하여 문서에 포함된 문장으로 요약
    - 장점: 생성요약보다 컴퓨팅 시간 적게 듦
    - 단점: 중요도가 가장 높은 한 개의 문장을 선택해서 많은 정보가 누락 

평가지표: `ROUGE Metric` - 모델의 요약 능력을 나타내는 자동화된 평가지표

In [6]:
#pip install git+https://github.com/SKT-AI/KoBART#egg=kobart
#!pip install transformers  #hugging face의 transformers 라이브러리 

In [7]:
test = df['text'].iloc[0]
test

'한국전통문화대학교 전통조경학과 2학년 학생들이 이재용 교수의 ‘전통식재론’ 수업의 일환으로 오는 4월과 5월에 걸쳐 고택정원 평면도를 작성한다 수업은 ‘고택정원의 과거와 현재의 차이’를 연구하기 위해 진행됐다 도면으로 작성되는 곳은 ▲구례 운조루 ▲논산 명재고택 ▲홍성 사운고택 ▲전주 학인당 ▲청주 고은리고택 ▲음성 잿말고택 ▲담양 창평장전이씨고택 지방 고택 7곳이다 현재 구례 운조루 전주 학인당 창평 장전이씨고택 논산 명재고택은 현황분석이 마무리된 상태이다 분석이 마무리된 고택의 현황자료들은 학생들이 직접 도면에 분석내용을 표현하고 지표를 만드는데 활용된다 그 외 수목 현황 표 및 고택 도감을 작성해 현재 모습의 정원을 기록한다 답사를 다녀온 전통조경학과 2학년 학생은 “직접 고택의 정원 현황도를 작성해 나가면서 단순히 책으로 보는 것보다 전통 정원에 대한 식재 구성을 파악하고 이해하는데 도움이 됐다”라며 “전문 장비들을 사용하면서 조사의 재미와 흥미를 느꼈고 직접 현황을 작성하고 남길 수 있어 의미있는 시간이었다”라고 소감을 전했다 3D 스캐너 활용 모습  라펜트\n\n\n\n타겟설치  라펜트\n\n\nGPS 활용 모습  라펜트 '

In [8]:
#모델과 문장 분리를 위한 토크나이저 불러오기
tokenizer = PreTrainedTokenizerFast.from_pretrained("ainize/kobart-news")
model = BartForConditionalGeneration.from_pretrained("ainize/kobart-news")

In [9]:
#요약하고자 하는 기사: test 
#토크나이저를 사용하여 뉴스기사 원문을 모델이 인식할 수 있는 토큰현태로 바꿔주기 
test = test.replace('\n', ' ')
raw_input_ids = tokenizer.encode(test)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

In [10]:
print(input_ids)
#텍스트가 토큰 단위로 쪼개어진 뒤 모두 id형태의 숫자로 바뀐 것을 확인
#"0": 문장 시작을 나타내는 토큰 id, "1": 문장 끝을 나타내는 id

[0, 0, 15274, 12126, 13241, 14912, 16765, 15686, 12178, 9085, 16382, 26430, 18631, 17507, 11908, 23977, 14143, 12126, 13241, 11466, 12080, 10341, 1543, 18141, 12024, 23518, 14834, 15483, 9120, 15503, 11786, 16795, 14068, 13146, 22036, 14223, 10586, 14704, 17248, 16728, 18141, 12005, 14143, 9102, 13146, 12141, 14865, 15320, 11863, 19734, 20262, 16027, 14541, 14191, 14185, 14434, 26480, 14121, 21102, 17248, 14343, 19105, 14948, 9170, 10329, 14251, 12178, 10396, 14948, 9608, 11211, 14248, 12080, 9102, 13146, 14948, 13710, 11280, 14031, 11914, 9102, 13146, 14948, 12126, 12258, 14208, 12037, 9770, 14948, 12681, 12258, 14068, 12005, 14268, 13146, 14948, 12013, 11280, 1700, 12088, 10504, 9102, 13146, 14948, 9766, 11747, 14411, 13455, 12074, 16984, 11682, 9102, 13146, 14915, 14068, 13146, 14235, 9114, 20162, 14475, 14112, 10329, 14251, 12178, 10396, 19813, 14208, 12037, 9770, 14411, 13455, 14137, 16984, 11682, 9102, 13146, 14396, 11211, 14248, 12080, 9102, 13146, 12005, 22836, 10952, 17920, 16

In [11]:
summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
summary1 = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)      #ainize/kobart-news 
summary1

"한국전통문화대학교 전통조경학과 2학년 학생들이 이재용 교수의 '전통식재론' 수업의 일환으로  ‘고택정원의 과거와 현재의 차이’를 연구하기 위해 오는 4월과 5월에 걸쳐 구례 운조루, 논산 명재고택,홍성 사운고택,전주 학인당,청주 고은리고택,청주 고은리고택,음성 잿말고택,담양 창평장전이씨고택 지방 고택 7곳에 대해 고택정원 평면도를 작성할 예정이다."

In [12]:
#모델과 문장 분리를 위한 토크나이저 불러오기
tokenizer = PreTrainedTokenizerFast.from_pretrained("gogamza/kobart-summarization")
model = BartForConditionalGeneration.from_pretrained("gogamza/kobart-summarization")

In [13]:
#요약하고자 하는 기사: test 
#토크나이저를 사용하여 뉴스기사 원문을 모델이 인식할 수 있는 토큰현태로 바꿔주기 
test = test.replace('\n', ' ')
raw_input_ids = tokenizer.encode(test)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
print(input_ids)

[0, 14188, 12126, 13241, 14912, 16765, 15686, 12178, 9085, 16382, 26430, 18631, 17507, 11908, 23977, 14143, 12126, 13241, 11466, 12080, 10341, 1543, 18141, 12024, 23518, 14834, 15483, 9120, 15503, 11786, 16795, 14068, 13146, 22036, 14223, 10586, 14704, 17248, 16728, 18141, 12005, 14143, 9102, 13146, 12141, 14865, 15320, 11863, 19734, 20262, 16027, 14541, 14191, 14185, 14434, 26480, 14121, 21102, 17248, 14343, 19105, 14948, 9170, 10329, 14251, 12178, 10396, 14948, 9608, 11211, 14248, 12080, 9102, 13146, 14948, 13710, 11280, 14031, 11914, 9102, 13146, 14948, 12126, 12258, 14208, 12037, 9770, 14948, 12681, 12258, 14068, 12005, 14268, 13146, 14948, 12013, 11280, 1700, 12088, 10504, 9102, 13146, 14948, 9766, 11747, 14411, 13455, 12074, 16984, 11682, 9102, 13146, 14915, 14068, 13146, 14235, 9114, 20162, 14475, 14112, 10329, 14251, 12178, 10396, 19813, 14208, 12037, 9770, 14411, 13455, 14137, 16984, 11682, 9102, 13146, 14396, 11211, 14248, 12080, 9102, 13146, 12005, 22836, 10952, 17920, 16568

In [14]:
summary_ids = model.generate(torch.tensor([input_ids]), bos_token_id = model.config.bos_token_id, eos_token_id = model.config.eos_token_id, length_penalty = 1.0, max_length = 300, min_length = 32, num_beams = 4)
summary2 = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)      #gogamza/kobart-summarization
summary2

'한국전통문화대학교 전통조경학과 2학년 학생들이 이재용 교수의 ‘전통식재론’ 수업의 일환으로 오는 4월과 5월에 걸쳐 고택정원의 과거와 현재의 차이’를 연구하기 위해 ‘고택정원의 과거와 현재의 차이’를 연구하기 위해 도면으로 작성되는 곳은 구례 운조루,논산 명재고택,홍성 사운고택,전주 학인당,전주 학인당,청주 고은리고택,음성 잿말고택,담양 창평장전이씨고택,담양 창평장전이씨고택,담양 창평장전이씨고택,담양 창평장전이씨고택,담양 창평장전이씨고택 등 7곳이다 현재 구례 운조루 전주 학인당 창평 장전이씨고택 논산 명재고택은 현황분석이 마무리된 상태이다 분석이 마무리된 고택의 현황자료들은 학생들이 직접 도면에 분석내용을 표현하고 지표를 만드는데 활용된다 그 외 수목 현황 표 및 고택 도감을 작성해 현재 모습의 정원을 기록한다 답사를 다녀온 전통조경학과 2학년 학생은 “직접 고택의 정원 현황도를 작성해 나가면서 단순히 책으로 보는 것보다 전통 정원에 대한 식재 구성을 파악하고 이해하는데 도움이 됐다”라며 “전문 장비들을 사용하면서 조사의 재미와 흥미를 느꼈고 직접 현황을 작성하고 남길 수'

In [15]:
#문서 요약 성능 
rouge = Rouge()
score1 = rouge.get_scores(summary1, test, avg=True)
score2 = rouge.get_scores(summary2, test, avg=True)

In [16]:
score1

{'rouge-1': {'r': 0.211864406779661,
  'p': 0.6756756756756757,
  'f': 0.322580641526743},
 'rouge-2': {'r': 0.13178294573643412,
  'p': 0.4722222222222222,
  'f': 0.20606060264903586},
 'rouge-l': {'r': 0.1440677966101695,
  'p': 0.4594594594594595,
  'f': 0.2193548350751301}}

In [17]:
score2

{'rouge-1': {'r': 0.7542372881355932,
  'p': 0.898989898989899,
  'f': 0.8202764927341842},
 'rouge-2': {'r': 0.7054263565891473,
  'p': 0.8425925925925926,
  'f': 0.7679324844907335},
 'rouge-l': {'r': 0.7542372881355932,
  'p': 0.898989898989899,
  'f': 0.8202764927341842}}

# 요약 모델

In [18]:
data = df.copy()

In [19]:
data["date"] = pd.to_datetime(data['date'])

In [20]:
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day

In [21]:
data.shape

(35126, 13)

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35126 entries, 0 to 35125
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   title        35126 non-null  object        
 1   sub-title    30484 non-null  object        
 2   publisher    35112 non-null  object        
 3   author       31925 non-null  object        
 4   date         35125 non-null  datetime64[ns]
 5   text         35126 non-null  object        
 6   keywords     35126 non-null  object        
 7   href         35126 non-null  object        
 8   numKeywords  35126 non-null  int64         
 9   news_id      35126 non-null  int64         
 10  year         35125 non-null  float64       
 11  month        35125 non-null  float64       
 12  day          35125 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(7)
memory usage: 3.5+ MB


In [23]:
data = data[data['year'] >= 2020]

In [24]:
data['year'].unique()

array([2022., 2020., 2023., 2021.])

In [25]:
data.shape

(7063, 13)

In [26]:
data.head(2)

,title,sub-title,publisher,author,date,text,keywords,href,numKeywords,news_id,year,month,day
1,2022년 올해의 정원식물은?,‘2022 대한민국 정원식물 전시·품평회’ 성료,라펜트,전지은,2022-11-07,‘2022 대한민국 정원식물 전시·품평회’를 9월 30일부터 10월 30일까지 개...,"['한국수목원정원관리원', '대한민국 정원식물 전시·품평회', '정원식물']",https://www.lafent.com/inews/news_view.html?ne...,3,131470,2022.0,11.0,7.0
2,‘골목길’ 주제 도시재생세미나 30일 개최,김선아 대표·김영욱 교수 주제발표,한국건설신문,선태규,2020-01-17,한국건축가협회 도시재생위원회위원장 김선아는 오는 30일 오후 6시 30분부터 8시 ...,"['한국건축가협회', '도시재생위원회', '도시재생', '골목길']",https://www.lafent.com/inews/news_view.html?ne...,4,125638,2020.0,1.0,17.0


In [27]:
data['publisher'].value_counts()

라펜트       6722
한국건설신문     232
기술인신문      100
기술사신문        7
Name: publisher, dtype: int64

In [28]:
data.isna().sum()

title            0
sub-title      109
publisher        2
author           7
date             0
text             0
keywords         0
href             0
numKeywords      0
news_id          0
year             0
month            0
day              0
dtype: int64

In [29]:
data.shape

(7063, 13)

In [30]:
#data에서 'summary' 칼럼 생성 
data["summary"] = ""

#용량이 너무 커서 데이터프레임 나누기 
data1 = data.iloc[:1000]
data2 = data.iloc[1001:2000]
data3 = data.iloc[2001:3000]
data4 = data.iloc[3001:4000]
data5 = data.iloc[4001:5000]
data6 = data.iloc[5001:6000]
data7 = data.iloc[6001:7000]
data8 = data.iloc[7001:]

In [31]:
#모델과 문장 분리를 위한 토크나이저 불러오기
tokenizer = PreTrainedTokenizerFast.from_pretrained("ainize/kobart-news")
model = BartForConditionalGeneration.from_pretrained("ainize/kobart-news")

In [32]:
def summarize(news, tokenizer, model):
    news = news.replace('\n', '')
    raw_input_ids = tokenizer.encode(news, max_length=1024, truncation=True)
    input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
    
    summary_ids = model.generate(torch.tensor([input_ids]), num_beams=4, max_length=512, eos_token_id=1)
    summary = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
    
    return summary 

In [35]:
for idx, df in enumerate([data1, data2, data3, data4, data5, data6, data7, data8], start=1):
    
    start_time = time.time()

    for index, row in tqdm(df.iterrows(), total=len(df)):  
        summary = summarize(row['text'], tokenizer, model)
        df.at[index, 'summary'] = summary

    end_time = time.time() 
    elapsed_time = end_time - start_time
    print(f"Dataframe {idx} 처리 완료. 경과 시간: {elapsed_time:.2f} 초")
    
    df.to_csv(f'요약_데이터_{idx}.csv', index=False)

100%|██████████| 1000/1000 [32:58<00:00,  1.98s/it]


Dataframe 1 처리 완료. 경과 시간: 1978.80 초


100%|██████████| 999/999 [37:45<00:00,  2.27s/it]  


Dataframe 2 처리 완료. 경과 시간: 2265.80 초


100%|██████████| 999/999 [39:35<00:00,  2.38s/it]  


...

Dataframe 6 처리 완료. 경과 시간: 2160.07 초


100%|██████████| 999/999 [34:16<00:00,  2.06s/it] 


Dataframe 7 처리 완료. 경과 시간: 2056.41 초


100%|██████████| 62/62 [02:04<00:00,  2.00s/it]

Dataframe 8 처리 완료. 경과 시간: 124.24 초


In [90]:
summarize = data['text']
summarize = list(summarize)
summarize

['‘2022 대한민국 정원식물 전시·품평회’를\xa0 9월 30일부터 10월 30일까지 개최됐다 국립세종수목원에서 개최된 ‘2022 대한민국 정원식물 전시·품평회’가 10월 30일 막을 내렸다 한국수목원정원관리원은 정원산업 활성화를 위해 ‘2022 대한민국 정원식물 전시·품평회’를\xa0 9월 30일부터 10월 30일까지 개최했다 이번 전시·품평회는 국화 그라스 침엽수 등 가을 정취를 느낄 수 있는 FW 시즌 정원식물 788품종을 한 자리에서 볼 수 있는 국내 최대 규모 전시다 품평회에서는 사전 진행된 전문가 및 국민 품평회에서 높은 평가를 받은 우수 정원식물 10종을 ‘올해의 정원식물’로 선정해 시상했다 ‘2022년 올해의 정원식물’ 자생식물 분야에서는 ▲‘향등골나물’이 종합 우수 ▲‘변산향유’는 색상 우수 ▲‘층실사초’ 질감 우수 ▲‘꼬랑사초’는 활용성 우수 ▲‘양뿔사초’는 신규성 우수로 각각 선정됐다 국내육성 품종 분야 ▲종합 우수는 ‘기린초 홍돌’ ▲색상 우수 ‘부채붓꽃 스노우윈드’ ▲질감 우수 ‘톱풀 루비스타’ ▲활용성 우수 ‘가우라 스타블러드’ ▲신규성 우수 ‘무궁화 꼬마’가 각각 수상했다 한수정은 이번 전시·품평회를 통해 발굴한 우수한 정원식물에 대해 정원 조성 정원장터 및 가든샵 판매 등을 통해 시장확산에 나설 계획이다 류광수 한수정 이사장은 “전시·품평회는 관상가치가 높은 정원식물을 정원 분야 관계자와 국민에게 선보이는 자리”라며 “우수한 정원식물을 발굴해 국내 정원시장 활성화에 힘쓰겠다”고 전했다 자생식물 분야\n\n종합 우수 ‘향등골나물’\n\n\n\n색상 우수 ‘변산향유’\n\n\n\n활용성 우수 ‘꼬랑사초’\n\n\n\n신규성 우수 ‘양뿔사초’\n\n\n\n\n국내육성 품종 분야\n\n종합 우수 ‘기린초 홍돌’\n\n\n\n색상 우수 ‘부채붓꽃 스노우윈드’\n\n\n\n활용성 우수 ‘가우라 스타블러드’\n\n\n\n신규성 우수 ‘무궁화 꼬마’ ',
 '한국건축가협회 도시재생위원회위원장 김선아는 오는 30일 오후 6시 30분부터 8시 3

In [99]:
summaries = []

for text in summarize:
    raw_input_ids = tokenizer.encode(text)
    input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
    summary_ids = model.generate(torch.tensor([input_ids]), num_beams=4,  max_length=512,  eos_token_id=1, early_stopping=True)
    summary = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
    summaries.append(summary)

29993


TypeError: exceptions must derive from BaseException